In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import xarray as xr

In [4]:
from neurodsp.plts.time_series import plot_time_series
from ripple_detection.core import gaussian_smooth
smoothing_sigma = 2 # seconds

In [5]:
from ecephys.data import channel_groups, paths
from ecephys.sglx_utils import load_timeseries
from ecephys.signal.timefrequency import parallel_spectrogram_welch
from ecephys.plot import plot_spectrogram, plot_hypnogram_overlay
from ecephys.scoring import load_visbrain_hypnogram

In [6]:
SUBJECT="Doppio"
CONDITION="REC-6-4"
START_TIME = 0
END_TIME = None
bin_path = paths.get_datapath(subject=SUBJECT, condition=CONDITION, data="lf.bin")

In [7]:
delta_range = (0.5, 4)
theta_range = (5, 10)

In [8]:
def get_spectrogram(sig):
    freqs, spg_times, spg = parallel_spectrogram_welch(sig.values, sig.fs)
    spg_times = spg_times + START_TIME
    spg = xr.DataArray(spg, dims=("frequency", "time", "channel"), coords={"frequency": freqs, "time": spg_times, "channel": sig.channel.values})
    return spg.median(dim="channel")
    
def _get_bandpower(spg, f_range, chans):
    bandpower = spg.sel(frequency=slice(*f_range)).sum(dim="frequency")
    bandpower.attrs['long_name'] = "{} Hz PSD".format(f_range)
    bandpower.attrs['units'] = 'uV^2/Hz'
    bandpower.attrs['chans'] = chans
    bandpower.attrs['freq_range'] = f_range
    
    return bandpower

## Cortex

In [9]:
cx_chans = channel_groups.superficial_ctx[SUBJECT]
cx_sig = load_timeseries(bin_path, cx_chans, start_time=START_TIME, end_time=END_TIME, xarray=True)

nChan: 385, nFileSamp: 18000019


In [10]:
cx_spg = get_spectrogram(cx_sig)

In [11]:
cx_theta = _get_bandpower(cx_spg, theta_range, cx_chans)
cx_delta = _get_bandpower(cx_spg, delta_range, cx_chans)

### Plot

In [12]:
plot_spectrogram(cx_spg.frequency, cx_spg.time, cx_spg, yscale='log')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
_, ax = plt.subplots(figsize=(24, 3))
plot_time_series(cx_spg.time.values, cx_delta.values, ax=ax, title="Cortical Delta", ylabel="Power")
#plot_hypnogram_overlay(hypno, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
_, ax = plt.subplots(figsize=(24, 3))
plot_time_series(cx_spg.time.values, cx_theta.values, ax=ax, title="Cortical Theta", ylabel="Power")
#plot_hypnogram_overlay(hypno, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
_, ax = plt.subplots(figsize=(24, 3))
plot_time_series(cx_spg.time.values, cx_theta.values / cx_delta.values, title="Cortical theta/delta ratio", ylabel="Ratio", ax=ax)
#plot_hypnogram_overlay(hypno, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Theta

In [16]:
sr_chans = channel_groups.stratum_radiatum_140um_to_200um[SUBJECT]
sr_sig = load_timeseries(bin_path, sr_chans, start_time=START_TIME, end_time=END_TIME, xarray=True)

nChan: 385, nFileSamp: 18000019


In [17]:
sr_spg = get_spectrogram(sr_sig)

In [18]:
sr_theta = _get_bandpower(sr_spg, theta_range, sr_chans)
sr_delta = _get_bandpower(sr_spg, delta_range, sr_chans)

### Plot

In [19]:
plot_spectrogram(sr_spg.frequency, sr_spg.time, sr_spg, yscale='log')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
_, ax = plt.subplots(figsize=(20, 3))
plot_time_series(sr_spg.time.values, sr_delta.values, title="SR Delta", ylabel="Power", ax=ax)
#plot_hypnogram_overlay(hypno, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
_, ax = plt.subplots(figsize=(20, 3))
plot_time_series(sr_spg.time.values, sr_theta.values, title="SR Theta", ylabel="Power", ax=ax)
#plot_hypnogram_overlay(hypno, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
_, ax = plt.subplots(figsize=(20, 3))
plot_time_series(sr_spg.time.values, sr_theta.values / sr_delta.values, title="SR theta/delta ratio", ylabel="Ratio", ax=ax)
#plot_hypnogram_overlay(hypno, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
_, ax = plt.subplots(figsize=(20, 3))
plot_time_series(sr_spg.time.values, np.log10(sr_theta.values) / np.log10(sr_delta.values), title="SR log(theta)/log(delta) ratio", ylabel="Ratio", ax=ax)
#plot_hypnogram_overlay(hypno, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
_, ax = plt.subplots(figsize=(24, 3))
plot_time_series(sr_spg.time.values, gaussian_smooth(sr_theta.values / sr_delta.values, smoothing_sigma, 1 / np.diff(sr_spg.time.values).mean()), title="Smoothed SR theta/delta ratio", ylabel="Ratio", ax=ax)
#plot_hypnogram_overlay(hypno, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Save

In [25]:
bandpower = xr.Dataset({'mpta_delta': cx_delta, 'mpta_theta': cx_theta, 'sr_delta': sr_delta, 'sr_theta': sr_theta})

In [26]:
bandpower.to_netcdf(paths.get_datapath(subject=SUBJECT, condition=CONDITION, data="lfp_bandpower.nc"))

In [27]:
bandpower.close()